In [2]:
import os
import json

from dotenv import load_dotenv
import requests

from openai import OpenAI, AzureOpenAI
from openai.types.chat import ChatCompletion

In [3]:
load_dotenv()

True

### [Function/Tool Calling] Connect LLM to external tools.

![tool-calling-flow](https://python.langchain.com/v0.2/assets/images/tool_calling_flow-ead8d93a8b69c88e3076457ed28f41ae.png)
Reference: https://platform.openai.com/docs/guides/function-calling

In [4]:
client = AzureOpenAI(
    api_version=os.getenv("OPENAI_API_VERSION"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

In [5]:
def get_chatgpt_response(messages: list[dict] = [], tools: list[dict] = None,
                         response_format: str = 'text') -> ChatCompletion:
    completion = client.chat.completions.create(
        model="gpt-4-0409-60k",
        messages=messages,
        response_format={'type': response_format},
        tools=tools,
        tool_choice='auto' if tools else None,
    )
    return completion

In [6]:
def google_search(query: str) -> list[dict]:
    """ Google search API

        Return a dict of custom search result, see more detail at
        https://developers.google.com/custom-search/v1/reference/rest/v1/Search#result
    """
    url = 'https://www.googleapis.com/customsearch/v1'
    params = {
        'q': query,
        'key': os.getenv("GOOGLE_SEARCH_API_KEY"),
        'cx': os.getenv("GOOGLE_SEARCH_ENGINE_ID"),
    }
    response = requests.get(url, params=params)
    results = response.json().get('items', [])
    return results

In [7]:
def get_google_search_content(search_target: str, verbose: bool = False) -> str:
    print(f"網路搜尋目標: {search_target}")
    content = "以下為已發生的事實：\n\n"
    search_results = google_search(search_target)
    for item in search_results:
        content += f"標題：{item['title']}\n摘要：{item['snippet']}\n\n"
    content += "請依照上述事實回答以下問題。\n"

    print(f'網路搜尋結果: \n{content}')
    return content

#### 1. 用 prompt，讓 AI 決定是否要網路搜尋

In [8]:
search_prompt_template = """
    如果我想知道以下這件事, 請確認是否需要網路搜尋才做得到?
    {user_question}

    如果需要, 請以下列 JSON 格式回答我, 除了 JSON 格式資料外, 不要加上額外資訊, 就算你知道答案, 也不要回覆:
    ``` {{"search": "Y", "keyword": "你建議的搜尋關鍵字"}} ```

    如果不需要, 請以下列 JSON 格式回答我:
    ``` {{"search": "N", "keyword": ""}} ```
"""

In [9]:
def get_response(user_question: str, assistant_prompt: str = '') -> dict:
    messages = [
        {"role": "system", "content": "請用 JSON 回覆"},
        *([{"role": "assistant", "content": assistant_prompt}] if assistant_prompt else []),
        {"role": "user", "content": search_prompt_template.format(user_question=user_question)},
    ]
    completion = get_chatgpt_response(messages, response_format='json_object')
    response: dict = json.loads(completion.choices[0].message.content)
    return response

In [10]:
# 情境一: 需要網路搜尋
question = '2023 金馬影后是誰?'
response = get_response(question)
response

{'search': 'Y', 'keyword': '2023 金馬影后'}

In [11]:
# 情境二: 可能不需要網路搜尋
question = '第一次世界大戰發生在哪一年?'
response = get_response(question)
response

{'search': 'N', 'keyword': ''}

In [12]:
# 情境二: 可能不需要網路搜尋
question = '第一個登上月球的人是誰?'
response = get_response(question)
response

{'search': 'N', 'keyword': ''}

In [15]:
# 情境三: 沒有前文脈絡
question = '那台灣呢?'
response = get_response(question)
response

{'search': 'Y', 'keyword': '台灣'}

In [16]:
# 情境四: 包含前文脈絡
assistant_prompt = '印度空污好嚴重'
user_question = '那台灣呢?'
response = get_response(user_question, assistant_prompt)
response

{'search': 'Y', 'keyword': 'Taiwan air pollution'}

#### 2. 問 AI 兩次，讓 AI 自己決定是否要網路搜尋

In [17]:
def get_response(user_question: str, system_prompt: str = '', assistant_prompt: str = '') -> str:
    # Step 1: 判斷是否需要網路搜尋
    messages = [
        {"role": "system", "content": "請用 JSON 回覆"},
        *([{"role": "assistant", "content": assistant_prompt}] if assistant_prompt else []),
        {"role": "user", "content": search_prompt_template.format(user_question=user_question)}
    ]
    completion = get_chatgpt_response(messages, response_format='json_object')
    response: dict = json.loads(completion.choices[0].message.content)

    # Step 2: 根據判斷結果, 決定是否進行網路搜尋
    if response.get('search') == 'Y':
        search_keyword = response.get('keyword')
        print(f"進行網路搜尋, 關鍵字: {search_keyword}")
        content = get_google_search_content(search_keyword)
    else:
        content = ""

    # Step 3: 以原始問題和網路搜尋結果, 組合成新的問題
    user_prompt = content + user_question

    # Step 4: 以新的問題, 請 ChatGPT 回答
    messages = [
        *([{"role": "system", "content": system_prompt}] if system_prompt else []),
        *([{"role": "assistant", "content": assistant_prompt}] if assistant_prompt else []),
        {"role": "user", "content": user_prompt},
    ]
    completion = get_chatgpt_response(messages)
    response = completion.choices[0].message.content
    return response

In [18]:
system_prompt = '使用繁體中文的小助理，請用 TEXT 回覆'
user_question = '第一個登上月球的人是誰?'
response = get_response(user_question, system_prompt)
response

'第一個登上月球的人是尼爾·阿姆斯壯(Neil Armstrong)，他於1969年7月20日踏上月球。'

In [19]:
system_prompt = '使用繁體中文的小助理，請用 TEXT 回覆'
assistant_prompt = '印度空污好嚴重'
user_question = '那台灣呢?'
response = get_response(user_question, system_prompt, assistant_prompt)
response

進行網路搜尋, 關鍵字: 台灣空氣污染程度
網路搜尋目標: 台灣空氣污染程度
網路搜尋結果: 
以下為已發生的事實：

標題：台灣空氣污染：實時空氣質量指數地圖
摘要：分享: “今天的空氣污染程度如何？查看實時空氣污染的數據- 全球超過100個國家。” https://aqicn.org/here/ ...

標題：空氣品質指標- 空氣品質監測網
摘要：將當日空氣中臭氧(O3)、細懸浮微粒(PM2.5)、懸浮微粒(PM10)、一氧化碳(CO)、二氧化硫(SO2) 及二氧化氮(NO2) 濃度等數值，以其對人體健康的影響程度，分別換算出不同污染物 ...

標題：全球空气质量指数（AQI）排名| IQAir
摘要：实时城市空气污染程度排名 小提示图标. 02:17 (当地时间). swiss flag image ... 82, 高雄市, 中国台湾. 38. 696K 关注. 83, 布达佩斯, 匈牙利. 38. 24.5K 关注.

標題：空氣品質監測網: 首頁
摘要：臺灣空品時空特徵 · 空品之最 · 認識空污感測物聯網. 專欄系列. 名人專欄. 常見 ... 當大氣中風速偏弱時，空氣流動較差，若有空氣污染物排放源，則使當地較易累積污染物。

標題：中国台湾空气质量指数（AQI）和中国台湾空气污染| IQAir
摘要：Jun 26, 2024 ... 获取您自己的监测仪，亲自测量您城市的空气吧。 成为数据提供者 · 了解数据提供者和数据来源. 实时中国台湾城市空气污染程度排名. #, city, 美国AQI. 1 ...

標題：空氣品質指標- 空氣品質改善維護資訊網
摘要：臭氧層保護在台灣 · 空氣品質淨化區 · 河川揚塵防制及改善推動 · 中元普度眉角多 ... 雖然不同的人受空氣污染影響的程度是取決於不同的因素，但是不同年齡的人都會受到 ...

標題：空氣污染指標(台灣) - 維基百科，自由的百科全書
摘要：空氣污染指標為依據監測資料將當日空氣中懸浮微粒(PM10)、二氧化硫(SO2)、二氧化氮(NO2)、一氧化碳 (CO) 及臭氧 (O3) 5種空氣污染物濃度數值，以其對人體健康的影響程度 ...

標題：初探臺灣空污、農業與社經條件不平等 - 風險社會與政策研究中心
摘要：Sep 11, 2021 ... 2021年在Science Advances的一

'根據您提供的信息，台灣也面臨空氣污染的問題。各種空氣品質監測網和資訊站點提供即時的空氣質量指數(AQI)和相關污染物濃度數據，例如臭氧(O3)、細懸浮微粒(PM2.5)、懸浮微粒(PM10)、一氧化碳(CO)、二氧化硫(SO2) 及二氧化氮(NO2)。台灣的各大城市，如高雄市，也出現在全球空氣質量指數排名中，顯示台灣的空氣質量受到廣泛關注。另外，不同的地區和不同的氣象狀況會影響空氣污染物的累積和分布。政府和相關機構也透過法規和政策，努力改善和維護空氣品質。'

#### 3. 用 tool，讓 AI 自己決定是否要網路搜尋
Reference: https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models#steps-to-invoke-a-function-call-using-chat-completions-api

In [20]:
def get_response(user_question: str, tools_spec: list[dict]) -> str:
    # Step 1: Prompt the model with content that may result in model selecting a tool to use.
    messages = [{"role": "user", "content": user_question}]

    completion = get_chatgpt_response(messages, tools_spec, response_format='text')
    assistant_response = completion.choices[0].message

    # Append the message to messages for next turn.
    messages.append(assistant_response)

    # Step 2: Determine if the response from the model includes a function call.
    tool_calls = completion.choices[0].message.tool_calls
    if tool_calls:
        # If true the model will return the name of the function to call and the argument(s).
        tool_call = tool_calls[0]
        tool_function = globals().get(tool_call.function.name, None)
        tool_function_args = json.loads(tool_call.function.arguments)

        # Step 3: Call the function and retrieve results. Append the results to the messages for the next turn.
        if tool_function:
            print(f'===== Calling function "{tool_call.function.name}" with arguments {tool_function_args}. =====')
            results = tool_function(**tool_function_args)
            # Note that messages with role 'tool' must be a response to a preceding message with 'tool_calls'.
            messages.append({"role": "tool", "content": results, "tool_call_id": tool_call.id, "name": tool_call.function.name})

            # Step 4: Invoke the chatgpt with the messages to get the response.
            completion = get_chatgpt_response(messages)
            response = completion.choices[0].message.content
        else:
            print(f"[Error] Function {tool_call.function.name} does not exist.")
    else:
        response = completion.choices[0].message.content

    return response


In [21]:
# Define a function specification for the function tool.
tools_spec = [
    {
        "type": "function",
        "function": {
            "name": "get_google_search_content",
            "description": "當遇到超過時間範圍的資料，利用 Google 取得搜尋結果",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_target": {
                        "type": "string",
                        "description": "要搜尋的關鍵字",
                    },
                },
                "required": ["search_target"],
            },
        },
    },
]

# Invoke the chatgpt to get the response.
user_question = '2024 奧運男子羽球雙打金牌得主是誰?'
response = get_response(user_question, tools_spec)
response

===== Calling function "get_google_search_content" with arguments {'search_target': "2024 Olympic men's badminton doubles gold medal winner"}. =====
網路搜尋目標: 2024 Olympic men's badminton doubles gold medal winner
網路搜尋結果: 
以下為已發生的事實：

標題：Badminton: Taiwan's Lee, Wang retain men's doubles gold | Reuters
摘要：15 hours ago ... PARIS, Aug 4 (Reuters) - Taiwan's Lee Yang and Wang Chi-Lin retained their Olympic badminton men's doubles title on Sunday, beating China's ...

標題：2024 Paris Olympic badminton pools revealed | NBC Olympics
摘要：Jul 15, 2024 ... One half of the reigning women's doubles gold medal ... In Tokyo, Axelsen dethroned 2016 Olympic champion Chen Long in the men's singles division.

標題：Paris 2024 badminton: All results, as Chinese Taipei's Lee Yang ...
摘要：16 hours ago ... Lee and Wang Chi-lin became the first pair in badminton history to win the men's doubles twice after an intense 21-17, 18-21, 21-19 victory over ...

標題：Chinese Taipei takes out China to defend men's doubles badm

'2024年奧運男子羽球雙打金牌得主是來自台灣的李洋和王齊麟。'

In [22]:
def get_pizza_info(pizza_name: str) -> str:
    pizza_info = [
        {
            "name": '夏威夷披薩',
            "price": "299",
            "description": "夏威夷披薩是以鳳梨、起司、番茄醬以及培根或火腿為配料的一種披薩。夏威夷披薩並非起源於夏威夷，而是由加拿大廚師山姆·潘諾普洛斯於1962年發明。"
        },
        {
            "name": '瑪格麗特披薩',
            "price": "319",
            "description": "瑪格麗特披薩是義大利的一種披薩餅，也是拿坡里披薩的代表。義大利傳說認為瑪格麗特披薩這個名字是來自於義大利瑪格麗特王后。由於瑪格麗特王后認為這種披薩上「綠色的羅勒、白色的莫札瑞拉起司、紅色的番茄醬好比義大利國旗」，對這種披薩十分鍾愛，因此用自己的名字命名了這款披薩。 "
        },
        {
            "name": '章魚小丸子披薩',
            "price": "346",
            "description": "和風章魚燒，從日本關西料理章魚燒中加入巧思變成經典口味的披薩"
        }
    ]
    return json.dumps(pizza_info)


# Define a function specification for the function tool.
tools_spec = [
    {
        "type": "function",
        "function": {
            "name": "get_pizza_info",
            "description": "這是一間披薩店，提供各種關於披薩的資訊",
            "parameters": {
                "type": "object",
                "properties": {
                    "pizza_name": {
                        "type": "string",
                        "description": "The name of the pizza, e.g. Salami",
                    },
                },
                "required": ["pizza_name"],
            },
        },
    },
]

# Invoke the chatgpt to get the response.
user_question = '請問夏威夷披薩多少錢？裡有什麼配料?'
response = get_response(user_question, tools_spec)
response

===== Calling function "get_pizza_info" with arguments {'pizza_name': 'Hawaiian'}. =====


'夏威夷披薩的價格為299元。它的配料包括鳳梨、起司、番茄醬以及培根或火腿。夏威夷披薩並非起源於夏威夷，而是由加拿大廚師山姆·檳安普羅斯於1962年發明。'